In [1]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display

In [2]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

In [13]:
import os
import glob
import pandas as pd
import PIL.Image
import shutil
from pathlib import Path

class ImageDataset:
    def __init__(self, parent_dir):
        self.img_files = sorted(glob.glob(os.path.join(parent_dir,"*.jpg")))
        
        # load labels
        labels = {}
        np_imgs = []
        pil_imgs = []
        for i, file in enumerate(self.img_files):
            labels[i] = {"filename": file}
            pil_img = PIL.Image.open(file)
            np_img = np.asarray(pil_img)
            pil_imgs.append(pil_img)
            np_imgs.append(np_img)
        self.pil_imgs = pil_imgs
        self.np_imgs = np_imgs
        
        self.labels = labels
            
    def get_size(self):
        return len(self.np_imgs)
    
    def get_np_img(self, i):
        return self.np_imgs[i]
    
    def get_pil_img(self, i):
        return self.pil_imgs[i]
    
    def get_label(self, i):
        throttle = self.labels[i]["throttle"]
        steer = self.labels[i]["steer"]
        return np.array([throttle,steer])
    
    def put_throttle(self, i, throttle):
        self.labels[i]["throttle"] = throttle
    
    def put_steer(self, i, steer):
        self.labels[i]["steer"] = steer
        
    def export(self):
        if os.path.exists("output_data"):
            shutil.rmtree("output_data")
        os.makedirs("output_data",exist_ok=True)
        with open("output_data/labels.csv", "w") as f:
            for i in range(len(self.np_imgs)):
                filename = str(Path(self.labels[i]["filename"]).name)
                if "steer" in self.labels[i].keys():
                    throttle = self.labels[i]["throttle"]
                    steer = self.labels[i]["steer"]
                    f.write("{},{},{}\n".format(filename,throttle,steer))

dataset = ImageDataset("input_data")

## Show original dataset

In [14]:
%matplotlib widget
import matplotlib.pyplot as plt

@widgets.interact(x=(0, dataset.get_size()-1, 1))
def show_image(x):
    print("No. {}".format(x))
    fig = plt.figure(figsize=(6,4))
    plt.imshow(dataset.get_np_img(x)) # show np image
    plt.show()
    #display(dataset.get_pil_img(x)) # show pil image

interactive(children=(IntSlider(value=112, description='x', max=224), Output()), _dom_classes=('widget-interac…

## Manually label steering input

In [15]:
%matplotlib widget
from ipywidgets import Output
import matplotlib.pyplot as plt

out = Output()
display(out)

x = 0

def show_image(x):
    print("No. {}".format(x))
    #print("Throttle: {}\n Steer: {}".format(dataset.get_label(x)[0],dataset.get_label(x)[1]))
    fig = plt.figure(figsize=(6,4))
    fig.canvas.mpl_connect('button_press_event', onclick)
    plt.imshow(dataset.get_np_img(x)) # show np image
    plt.show()
    #display(dataset.get_pil_img(x)) # show pil image

@out.capture(clear_output=True)
def onclick(event):
    global x
    try:
        print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
              ('double' if event.dblclick else 'single', event.button,
               event.x, event.y,
               event.xdata, event.ydata))
        w = len(dataset.get_np_img(x)[0])
        manually_labeled_throttle = 0.5
        manually_labeled_steer = -1*(event.xdata-(w/2))/(w/2)
        print(manually_labeled_steer)
        dataset.put_throttle(x, manually_labeled_throttle)
        dataset.put_steer(x, manually_labeled_steer)
    except TypeError:
        print("type error: skipping image {}".format(x))
    if x==dataset.get_size()-1:
        print("done")
        dataset.export()
    else:
        x += 1
        show_image(x)

with out:
    show_image(x)

Output()

In [11]:
"steer" in dataset.labels[0].keys()

True